In [3]:
import pandas as pd, torch, torch.nn as nn, pickle
from sklearn.model_selection import train_test_split
import glob, os, timeit
import torch.nn.functional as F

In [4]:
torch.cuda.is_available()

True

In [5]:
all_data = pd.read_csv('megasequence-to-cancer.csv')

In [6]:
all_data.head()

,Sample ID,sequence,Cancer type
0,Sample_0025,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
1,Sample_0026,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
2,Sample_0027,AACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACC...,Normal
3,Sample_0028,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal
4,Sample_0029,NNNTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal


In [7]:
%load_ext Cython

In [8]:
%%cython

import numpy as np
np.get_include() # do we need this on colab? 
cimport cython
cimport numpy as np

# Adding Z here so we can encode the separator
cdef dict bases={ 'A':<int>0, 'C':<int>1, 'G':<int>2, 'T':<int>3, 'Z':<int>4} 

@cython.boundscheck(False)
def one_hot( str string ):
    cdef np.ndarray[np.float32_t, ndim=2] res = np.zeros( (5,len(string)), dtype=np.float32 )
    cdef int j
    for j in range(len(string)):
        if string[j] in bases: # bases can be 'N' signifying missing: this corresponds to all 0 in the encoding
            res[ bases[ string[j] ], j ]=float(1.0)
    return(res)


In [ ]:
genome = pickle.load(open('hg19.pickle', 'rb'))

In [9]:
# In the first step we will split the data in training and remaining dataset
data_train, data_rem = train_test_split(all_data, train_size=0.8, random_state=543)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
data_validation, data_test = train_test_split(data_rem, test_size=0.5, random_state=82)

In [10]:
stuff = data_train
stuff['size'] = stuff['sequence'].apply(lambda row: len(row))
stuff.describe()

/tmp/ipykernel_222240/943087518.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stuff['size'] = stuff['sequence'].apply(lambda row: len(row))


,size
count,3.610000e+02
mean,1.614851e+07
std,1.231679e+07
min,8.181000e+03
25%,6.239024e+06
50%,1.354215e+07
75%,2.348382e+07
max,9.384835e+07


In [11]:
# for chip-seq data: also shuffling nucleotides can be done to keep the GC content the same as positive example
# because sequencing has biases with GC content and this would be a way to "fix it"
class BedPeaksDataset(torch.utils.data.IterableDataset):

    def __init__(self, data_set):
        super(BedPeaksDataset, self).__init__()
        self.atac_data = data_set

    def __iter__(self): 
        for i,row in enumerate(self.atac_data.itertuples()):
            seq = row.sequence
            value = np.float32(1)
            print(row)
            if row._3 == "Normal":
                value = np.float32(0)
            yield(one_hot(seq), value) # positive example

train_dataset = BedPeaksDataset(data_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1000, num_workers = 0)

#validation_dataset = BedPeaksDataset(data_validation)
#validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=1000, num_workers=0)

#test_dataset = BedPeaksDataset(data_test)
#test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, num_workers=0)

data_train

,Sample ID,sequence,Cancer type,size
73,Sample_0243,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal,7219593
289,Sample_0893,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Leukemia,62366209
196,Sample_0544,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Leukemia,43331138
316,Sample_1171,ACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCC...,Normal,15393051
256,Sample_0832,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,The Yellow fever,4825374
...,...,...,...,...
361,Sample_1255,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal,4743539
132,Sample_0302,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal,29292305
49,Sample_0219,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal,25674355
103,Sample_0273,CCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCTAACCCT...,Normal,10586725


In [12]:
def run_one_epoch(train_flag, dataloader, model, optimizer, device="cuda"):

    torch.set_grad_enabled(train_flag)
    model.train() if train_flag else model.eval() 

    losses = []
    accuracies = []

    for (x,y) in dataloader: # collection of tuples with iterator

        (x, y) = ( x.to(device), y.to(device) ) # transfer data to GPU
        print('a')

        output = model(x) # forward pass
        print('b')
        output = output.squeeze(1) # remove spurious channel dimension
        print('c')
        loss = F.binary_cross_entropy_with_logits( output, y ) # numerically stable
        print('d')

        if train_flag: 
            loss.backward() # back propagation
            print('e')
            optimizer.step()
            print('f')
            optimizer.zero_grad()

        print('g')
        losses.append(loss.detach().cpu().numpy())
        accuracy = torch.mean( ( (output > .5) == (y > .5) ).float() )
        accuracies.append(accuracy.detach().cpu().numpy())
        print(losses[-1], accuracies[-1])
    
    return( np.mean(losses), np.mean(accuracies) )

In [13]:
def train_model(model, train_data, validation_data, epochs=100, patience=10, batch_size=1000, verbose = True):
    """
    Train a 1D CNN model and record accuracy metrics.
    """
    # Move the model to the GPU here to make it runs there, and set "device" as above
    # TODO CODE
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # 1. Make new BedPeakDataset and DataLoader objects for both training and validation data.
    # TODO CODE
    train_dataset = BedPeaksDataset(train_data)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers = 3, timeout=600)
    validation_dataset = BedPeaksDataset(validation_data)
    validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, num_workers = 3, timeout=600)

    # 2. Instantiates an optimizer for the model. 
    # TODO CODE
    optimizer = torch.optim.Adam(model.parameters(), amsgrad=True)

    # 3. Run the training loop with early stopping. 
    # TODO CODE
    train_accs = []
    val_accs = []
    train_losses = []
    val_losses = []
    patience_counter = patience
    best_val_loss = np.inf
    
    
    # Get a list of all the file paths that ends with .txt from in specified directory
    fileList = glob.glob('./checkpoints/model_checkpoint*.pt')
    # Iterate over the list of filepaths & remove each file.
    for filePath in fileList:
        try:
            os.remove(filePath)
        except:
            print("Error while deleting file : ", filePath)
    
    
    check_point_filename = './checkpoints/model_checkpoint.pt' # to save the best model fit to date
    local_min_counter = 0
    for epoch in range(epochs):
        start_time = timeit.default_timer()
        train_loss, train_acc = run_one_epoch(True, train_dataloader, model, optimizer, device)
        val_loss, val_acc = run_one_epoch(False, validation_dataloader, model, optimizer, device)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        if val_loss < best_val_loss: 
            if patience_counter < patience:
                local_min_counter = local_min_counter + 1
            torch.save(model.state_dict(), f'./checkpoints/model_checkpoint-{local_min_counter}.pt')
            best_val_loss = val_loss
            patience_counter = patience
        else: 
            patience_counter -= 1
            if patience_counter <= 0: 
                model.load_state_dict(torch.load(f'./checkpoints/model_checkpoint-{local_min_counter}.pt')) # recover the best model so far
                break
        elapsed = float(timeit.default_timer() - start_time)
        print("Epoch %i took %.2fs. Train loss: %.4f acc: %.4f. Val loss: %.4f acc: %.4f. Patience left: %i" % 
              (epoch+1, elapsed, train_loss, train_acc, val_loss, val_acc, patience_counter ))

    # 4. Return the fitted model (not strictly necessary since this happens "in place"), train and validation accuracies.
    # TODO CODE
    return model, train_accs, val_accs, train_losses, val_losses, validation_dataloader

In [14]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

In [15]:
class LSTM(nn.Module):

    def __init__(self, 
                 n_output_channels = 1, 
                 n_hidden = 32, 
                 dropout = 0.2,
                 n_fc = 1,
                 lstm_hidden=10):
        
        super(LSTM, self).__init__()
        self.lstm = nn.Sequential(nn.GRU(5, lstm_hidden, batch_first=True, dropout=dropout))
        
        fc_layers = [nn.Linear(lstm_hidden, n_hidden)]
        for i in range(n_fc-1):
            fc_layers += [ nn.Dropout(dropout),
                          nn.ELU(inplace=True),
                          nn.Linear(n_hidden, n_hidden)
            ]
        fc_layers += [nn.Dropout(dropout),
                      nn.ELU(inplace=True),
                      nn.Linear(n_hidden, n_output_channels)
        ]
        self.dense_net = nn.Sequential(*fc_layers)

    def forward(self, x):
        
        #print(x.size())
        # switch sequence and channels and send to LSTM
        #net, (hn, cn) = self.lstm(x.swapaxes(1, 2))
        net, hn = self.lstm(x.swapaxes(1, 2))
        #print(net.size())
        net = net[:, -1, :]
        #print(net.size())
        net = net.view(net.size(0), -1)
        #print(net.size())
        net = self.dense_net(net)
        print(net.size())
        return(net)

In [ ]:
torch.cuda.empty_cache()

lstm_model = LSTM(dropout=.28, n_hidden=32, n_fc=2, lstm_hidden=2)

lstm_model, train_accs, val_accs, train_losses, val_losses, lstm_validation_datloader = train_model(lstm_model, data_train
                                                                         , data_validation, epochs=300
                                                                         , patience=20, batch_size=1)

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.28 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...


torch.Size([1, 1])
b
c
d
e
f
g
0.75025254 1.0
a


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




torch.Size([1, 1])
b
c
d


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




e
f
g
0.71687573 1.0
a
torch.Size([1, 1])
b
c
d


In [ ]:
%debug